In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
import os
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
from itertools import product, combinations
import math
import warnings
# Import some libraries
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import georasters as gr

In [2]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Leiden','Tashkent, Uzbekistan','Providence, Rhode Island, United States']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)

park_entry_point_merge = 0 # ! Be careful with siplyfying topology with this, can alter the scores quite a bit.
grid_block_size = 200 # in meters2 rounded to 100m
nearest_node_iterations = 25
block_combinations = 1000
block_network = 250000

In [3]:
# Block 1 Extract city boundaries, download grids in cities' country

warnings.filterwarnings("ignore")
bound_df = ox.geocoder.geocode_to_gdf(cities)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')

start_time = time.time()

# Get unique ISO countries, so all country-grids are only loaded once (i.e. Philadelphia, Denver and Washington DC in the USA)
C = pd.DataFrame(Countries)
start_time = time.time()
iso_countries = []
print('required WorldPoP files (execute this per country in the Jupyter terminal)')
for i in range(len(cities)): 
    countries = bound_df['display_name'][i].rsplit(',')[-1][1:]
    iso = C[C['name'] == countries].iloc[0,1]
    if iso not in iso_countries:
        iso_countries.append(iso)
        
        # Search for data
        products = Product(iso)
        Results = products.description_contains('people per grid-cell 2020')
        list1 = []
        for p in Results:
            prints = '%s/%s\t%s\t%s' % (p.idx, p.country_name,p.dataset_name,p.path)
            list1.append(prints)
        print('wpgpDownload download -i',iso,'--id',list1[0].split("\t")[0].split('/')[0])

print('')
blocks = []
for i in iso_countries:
    # Get the country by saved path ()
    # You can also use sys.path: path = sys.path[0] +'\\' + str('ISL').lower() + '_ppp_2020.tif'
    # The sys.path is the path Python saves the python file automatically before you
    path = 'D:\\Dumps\\WorldPoP_Grids\\' + i.lower() + '_ppp_2020.tif'
    block = gr.from_file(path)
    blocks.append(block)
    print(i,'extracted', round((time.time() - start_time)/60,2),'mns')
bound_df

required WorldPoP files (execute this per country in the Jupyter terminal)
wpgpDownload download -i NLD --id 5132
wpgpDownload download -i UZB --id 5222
wpgpDownload download -i USA --id 4983

NLD extracted 0.01 mns
UZB extracted 0.06 mns
USA extracted 1.31 mns


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((4.43887 52.15326, 4.43890 52.15321, ...",52.184626,52.118952,4.524072,4.438865,282019494,relation,295092,52.151816,4.481109,"Leiden, South Holland, Netherlands",boundary,administrative,0.729246
1,"POLYGON ((69.12171 41.26591, 69.12184 41.26559...",41.398638,41.163421,69.407783,69.121710,337169612,relation,2216724,41.312336,69.278708,"Tashkent, 100000, Uzbekistan",boundary,administrative,0.828581
2,"POLYGON ((-71.47267 41.83711, -71.47264 41.837...",41.861801,41.772414,-71.373614,-71.472667,281824009,relation,191210,41.823989,-71.412834,"Providence, Providence County, Rhode Island, U...",boundary,administrative,1.076668


In [4]:
# Block 2 population grids extraction
start_time = time.time()
clips = []
grids = []
print('300m resolution grids extraction')
for i in range(len(cities)):
    iso = C[bound_df['display_name'][i].rsplit(',')[-1][1:] == C['name']].iloc[0,1]
    contains = [j for j, x in enumerate(iso_countries) if x == iso][0]
    
    # Clip the city from the country
    clipped = blocks[contains].clip(bound_df['geometry'][i])
    clipped = clipped[0].to_geopandas()
    
    # Get dissolvement_key for dissolvement. 
    clipped['row3'] = np.floor(clipped['row']/(grid_block_size/100)).astype(int)
    clipped['col3'] = np.floor(clipped['col']/(grid_block_size/100)).astype(int)
    clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)
    clips.append(clipped)
        
    # Dissolve into block by block grids
    popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

    # Get those grids populations and area. Only blocks with population and full blocks
    popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
    popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
    popgrid = popgrid[popgrid['population'] > 0]
    popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]
    
    # Get centroids and coords
    popgrid['centroid'] = popgrid['geometry'].centroid
    popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    popgrid = popgrid.reset_index()
    
    minx = popgrid.bounds['minx']
    maxx = popgrid.bounds['maxx']
    miny = popgrid.bounds['miny']
    maxy = popgrid.bounds['maxy']

    # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
    # I therefore create one polygon
    Poly = []
    for k in range(len(popgrid)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    popgrid['geometry'] = Poly
    
    grids.append(popgrid)
    
    print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    
grids[0]

300m resolution grids extraction
Leiden 0.02 mns
Tashkent 0.16 mns
Providence 0.18 mns


,dissolve_key,geometry,row3,col3,population,area_m,centroid,centroid_m,grid_lon,grid_lat
0,0-38,"POLYGON ((4.50208 52.18542, 4.50375 52.18542, ...",0,38,50,21129,POINT (4.50292 52.18458),POINT (602748.237 5782633.106),602748.237108,5.782633e+06
1,1-35,"POLYGON ((4.49708 52.18375, 4.49875 52.18375, ...",1,35,82,21129,POINT (4.49792 52.18292),POINT (602410.256 5782440.672),602410.256090,5.782441e+06
2,1-36,"POLYGON ((4.49875 52.18375, 4.50042 52.18375, ...",1,36,100,21129,POINT (4.49958 52.18292),POINT (602524.197 5782443.027),602524.197091,5.782443e+06
3,1-37,"POLYGON ((4.50042 52.18375, 4.50208 52.18375, ...",1,37,102,21129,POINT (4.50125 52.18292),POINT (602638.138 5782445.384),602638.138070,5.782445e+06
4,1-38,"POLYGON ((4.50208 52.18375, 4.50375 52.18375, ...",1,38,99,21129,POINT (4.50292 52.18292),POINT (602752.079 5782447.745),602752.079028,5.782448e+06
...,...,...,...,...,...,...,...,...,...,...
1085,9-42,"POLYGON ((4.50875 52.17042, 4.51042 52.17042, ...",9,42,155,21136,POINT (4.50958 52.16958),POINT (603238.711 5780974.323),603238.711205,5.780974e+06
1086,9-43,"POLYGON ((4.51042 52.17042, 4.51208 52.17042, ...",9,43,155,21136,POINT (4.51125 52.16958),POINT (603352.686 5780976.697),603352.686145,5.780977e+06
1087,9-44,"POLYGON ((4.51208 52.17042, 4.51375 52.17042, ...",9,44,155,21136,POINT (4.51292 52.16958),POINT (603466.661 5780979.073),603466.661064,5.780979e+06
1088,9-45,"POLYGON ((4.51375 52.17042, 4.51542 52.17042, ...",9,45,151,21136,POINT (4.51458 52.16958),POINT (603580.636 5780981.452),603580.635961,5.780981e+06


In [5]:
# Block 3 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type="all", buffer_dist = (np.max(thresholds)+1000))
    graphs.append(graph)
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    road_nodes.append(road_node)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    road_edges.append(road_edge)
    
    # Get only necessary road connections columns for network performance
    road_con = road_edge[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
road_edges[0]

Leiden done 0.52 mns
Tashkent done 1.49 mns
Providence done 2.0 mns


,from,to,key,osmid,name,highway,oneway,length,geometry,maxspeed,...,bridge,service,lanes,width,access,junction,ref,area,est_width,geometry_m
0,30680610,259603511,30680610-259603511,4815575,Punterwerf,pedestrian,False,4.701,"LINESTRING (4.51251 52.17519, 4.51245 52.17518)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (603426.054 5781602.264, 603421.606..."
1,30680610,30680614,30680610-30680614,4815575,Punterwerf,pedestrian,False,42.589,"LINESTRING (4.51251 52.17519, 4.51257 52.17521...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (603426.054 5781602.264, 603429.794..."
2,30680610,664050795,30680610-664050795,52109126,NaN,footway,False,22.474,"LINESTRING (4.51251 52.17519, 4.51249 52.17509...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (603426.054 5781602.264, 603425.033..."
3,30680614,1908503936,30680614-1908503936,180401870,Punterwerf,footway,False,8.229,"LINESTRING (4.51311 52.17515, 4.51308 52.17508)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (603467.202 5781598.271, 603465.324..."
4,30680614,30680610,30680614-30680610,4815575,Punterwerf,pedestrian,False,42.589,"LINESTRING (4.51311 52.17515, 4.51263 52.17522...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (603467.202 5781598.271, 603433.840..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77598,9811404122,9811402815,9811404122-9811402815,1068849309,NaN,path,False,54.224,"LINESTRING (4.48724 52.13015, 4.48717 52.13009...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (601800.447 5776557.371, 601795.931..."
77599,9811404123,9811404128,9811404123-9811404128,1068849310,NaN,path,False,21.843,"LINESTRING (4.48788 52.12981, 4.48788 52.12980...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (601844.999 5776520.144, 601845.421..."
77600,9811404128,8272023055,9811404128-8272023055,889892828,NaN,path,False,11.290,"LINESTRING (4.48772 52.12967, 4.48779 52.12957)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (601834.615 5776503.953, 601839.731..."
77601,9811404128,8272023057,9811404128-8272023057,889892828,NaN,path,False,11.626,"LINESTRING (4.48772 52.12967, 4.48761 52.12975)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (601834.615 5776503.953, 601827.072..."


In [6]:
# Block 4 city greenspace
parks_in_range = list()
for i in cities:
    gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
    gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
    greenspace = gdf.reset_index()    
    warnings.filterwarnings("ignore")
    
    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')
    
    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
    print(i, 'done')
    parks = parks.reset_index()
    parks_in_range.append(parks)
parks_in_range[0]

Leiden done
Tashkent, Uzbekistan done
Providence, Rhode Island, United States done


,components,geometry,element_type,osmid,leisure,nodes,garden:style,name,website,wikidata,...,survey:date,construction,opening_hours,ways,alt_name,park,type,note,geometry_w_buffer,geom buffer diff
0,0,"POLYGON ((4.44671 52.12694, 4.44616 52.12719, ...",way,6315751,park,"[45362626, 8911972296, 45366408, 5534495534, 5...",english,Burgemeester Berkhoutpark,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.44535 52.12789, 4.44535 52.12789, ...","POLYGON ((4.44535 52.12789, 4.44536 52.12791, ..."
1,1,"POLYGON ((4.49495 52.13246, 4.49484 52.13254, ...",way,6315783,park,"[5376777171, 7597508808, 4675237765, 937906360...",None,Polderpark Cronesteyn,http://www.polderparkcronesteyn.nl/,Q3926591,...,None,None,None,None,None,None,None,None,"POLYGON ((4.49292 52.14255, 4.49398 52.14271, ...","POLYGON ((4.49398 52.14271, 4.49502 52.14300, ..."
2,2,"POLYGON ((4.42759 52.14521, 4.42722 52.14472, ...",way,6315791,park,"[573405841, 45446500, 45441718, 45438110, 4473...",None,Zuidwijk,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.42793 52.14529, 4.42794 52.14527, ...","POLYGON ((4.42794 52.14527, 4.42795 52.14525, ..."
3,3,"POLYGON ((4.50849 52.14706, 4.50855 52.14715, ...",way,6315792,park,"[1109479882, 1109569716, 1109462158, 454981039...",None,Park De Bult,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.50666 52.14613, 4.50616 52.14652, ...","POLYGON ((4.50616 52.14652, 4.50614 52.14654, ..."
4,4,"POLYGON ((4.45893 52.14864, 4.45943 52.14904, ...",way,6315800,park,"[45464044, 45465644, 45467524, 45468854, 45469...",None,Ter Wadding,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.45859 52.14872, 4.45861 52.14874, ...","POLYGON ((4.45861 52.14874, 4.45862 52.14876, ..."
5,5,"POLYGON ((4.53240 52.15586, 4.53221 52.15596, ...",way,6315823,park,"[1126386633, 9082269898, 1126403033, 908226989...",None,Park De Houtkamp,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.52835 52.15942, 4.52832 52.15943, ...","POLYGON ((4.52832 52.15943, 4.52829 52.15944, ..."
6,6,"POLYGON ((4.48789 52.17332, 4.48777 52.17329, ...",way,6315850,park,"[45564010, 45564397, 45571154, 45573534, 45576...",None,Heempark,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.48824 52.17326, 4.48823 52.17324, ...","POLYGON ((4.48823 52.17324, 4.48821 52.17322, ..."
7,7,"MULTIPOLYGON (((4.46462 52.17703, 4.46521 52.1...",way,6315853,park,"[45575107, 45576801, 45577188, 45577598, 77682...",None,Endegeest,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.46294 52.17649, 4.46294 52.17651, ...","POLYGON ((4.46294 52.17651, 4.46294 52.17653, ..."
8,8,"POLYGON ((4.50869 52.17658, 4.50845 52.17646, ...",way,6315862,park,"[45576310, 5050734260, 5050734256, 5050734249,...",None,Merenwijkpark,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.50356 52.17886, 4.50340 52.17891, ...","POLYGON ((4.50340 52.17891, 4.50336 52.17892, ..."
9,9,"POLYGON ((4.48058 52.17422, 4.47757 52.17511, ...",way,6325515,park,"[45568346, 45570466, 7149875869, 45572533, 455...",None,Leidse Hout,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((4.48079 52.17404, 4.48076 52.17403, ...","POLYGON ((4.48076 52.17403, 4.48073 52.17402, ..."


In [7]:
# Block 5 park entry points
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry']], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_proot2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_proot3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_proot5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Leiden 0.0 % done 0.0  mns
Leiden 94.3 % done 0.12  mns
Leiden 100 % done 0.17  mns
Tashkent 0.0 % done 0.17  mns
Tashkent 60.2 % done 0.35  mns
Tashkent 100 % done 0.47  mns
Providence 0.0 % done 0.48  mns
Providence 44.2 % done 0.55  mns
Providence 88.5 % done 0.62  mns
Providence 100 % done 0.65  mns


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5
0,45362627,52.126866,4.446763,3,NaN,NaN,POINT (4.44676 52.12687),POINT (599037.460 5776135.853),45362627,0,...,5.776136e+06,"POLYGON ((4.44671 52.12694, 4.44616 52.12719, ...","POLYGON ((4.44616 52.12719, 4.44560 52.12746, ...",56338.095251,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759
1,45362808,52.126915,4.446965,3,NaN,NaN,POINT (4.44696 52.12692),POINT (599051.185 5776141.612),45362808,0,...,5.776142e+06,"POLYGON ((4.44671 52.12694, 4.44616 52.12719, ...","POLYGON ((4.44616 52.12719, 4.44560 52.12746, ...",56338.095251,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759
2,45368876,52.128427,4.450258,3,NaN,NaN,POINT (4.45026 52.12843),POINT (599273.226 5776314.248),45368876,0,...,5.776314e+06,"POLYGON ((4.44671 52.12694, 4.44616 52.12719, ...","POLYGON ((4.44616 52.12719, 4.44560 52.12746, ...",56338.095251,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759
3,45369775,52.128619,4.445725,4,NaN,NaN,POINT (4.44573 52.12862),POINT (598962.544 5776329.389),45369775,0,...,5.776329e+06,"POLYGON ((4.44671 52.12694, 4.44616 52.12719, ...","POLYGON ((4.44616 52.12719, 4.44560 52.12746, ...",56338.095251,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759
4,45372489,52.129268,4.449065,4,NaN,NaN,POINT (4.44906 52.12927),POINT (599189.685 5776406.161),45372489,0,...,5.776406e+06,"POLYGON ((4.44671 52.12694, 4.44616 52.12719, ...","POLYGON ((4.44616 52.12719, 4.44560 52.12746, ...",56338.095251,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,5615701111,52.196118,4.535691,1,NaN,NaN,POINT (4.53569 52.19612),POINT (604961.573 5783962.920),5615701111,52,...,5.783963e+06,"POLYGON ((4.51057 52.19572, 4.51075 52.19584, ...","POLYGON ((4.53041 52.19301, 4.52994 52.19334, ...",394619.384869,7.295753e+06,0.054089,7.426103,2.725088,1.950984,1.493318
1151,8034209428,52.197458,4.507242,1,NaN,NaN,POINT (4.50724 52.19746),POINT (603014.167 5784071.092),8034209428,52,...,5.784071e+06,"POLYGON ((4.51057 52.19572, 4.51075 52.19584, ...","POLYGON ((4.51450 52.19692, 4.51438 52.19649, ...",202042.771960,7.295753e+06,0.027693,3.802120,1.949903,1.560781,1.306186
1152,8034209431,52.198292,4.506893,1,NaN,NaN,POINT (4.50689 52.19829),POINT (602988.374 5784163.351),8034209431,52,...,5.784163e+06,"POLYGON ((4.51057 52.19572, 4.51075 52.19584, ...","POLYGON ((4.51416 52.19776, 4.51404 52.19732, ...",146574.297629,7.295753e+06,0.020090,2.758293,1.660811,1.402427,1.224977
1153,8034209488,52.198647,4.507683,1,NaN,NaN,POINT (4.50768 52.19865),POINT (603041.582 5784204.012),8034209488,52,...,5.784204e+06,"POLYGON ((4.51057 52.19572, 4.51075 52.19584, ...","POLYGON ((4.51495 52.19811, 4.51483 52.19768, ...",161157.024852,7.295753e+06,0.022089,3.032716,1.741470,1.447473,1.248436


In [8]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5,geometry_m_buffer,components,comp_centroid,centr_dist
0,45362627,52.126866,4.446763,3,NaN,NaN,POINT (4.44676 52.12687),POINT (599037.460 5776135.853),45362627,0,...,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759,POLYGON EMPTY,0,POINT (4.44676 52.12687),0.0
1,45362808,52.126915,4.446965,3,NaN,NaN,POINT (4.44696 52.12692),POINT (599051.185 5776141.612),45362808,0,...,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759,POLYGON EMPTY,1,POINT (4.44696 52.12692),0.0
2,45368876,52.128427,4.450258,3,NaN,NaN,POINT (4.45026 52.12843),POINT (599273.226 5776314.248),45368876,0,...,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759,POLYGON EMPTY,2,POINT (4.45026 52.12843),0.0
3,45369775,52.128619,4.445725,4,NaN,NaN,POINT (4.44573 52.12862),POINT (598962.544 5776329.389),45369775,0,...,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759,POLYGON EMPTY,3,POINT (4.44573 52.12862),0.0
4,45372489,52.129268,4.449065,4,NaN,NaN,POINT (4.44906 52.12927),POINT (599189.685 5776406.161),45372489,0,...,5.633810e+04,1.000000,1.060192,1.029656,1.019675,1.011759,POLYGON EMPTY,4,POINT (4.44906 52.12927),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,5615701111,52.196118,4.535691,1,NaN,NaN,POINT (4.53569 52.19612),POINT (604961.573 5783962.920),5615701111,52,...,7.295753e+06,0.054089,7.426103,2.725088,1.950984,1.493318,POLYGON EMPTY,1150,POINT (4.53569 52.19612),0.0
1151,8034209428,52.197458,4.507242,1,NaN,NaN,POINT (4.50724 52.19746),POINT (603014.167 5784071.092),8034209428,52,...,7.295753e+06,0.027693,3.802120,1.949903,1.560781,1.306186,POLYGON EMPTY,1151,POINT (4.50724 52.19746),0.0
1152,8034209431,52.198292,4.506893,1,NaN,NaN,POINT (4.50689 52.19829),POINT (602988.374 5784163.351),8034209431,52,...,7.295753e+06,0.020090,2.758293,1.660811,1.402427,1.224977,POLYGON EMPTY,1152,POINT (4.50689 52.19829),0.0
1153,8034209488,52.198647,4.507683,1,NaN,NaN,POINT (4.50768 52.19865),POINT (603041.582 5784204.012),8034209488,52,...,7.295753e+06,0.022089,3.032716,1.741470,1.447473,1.248436,POLYGON EMPTY,1153,POINT (4.50768 52.19865),0.0


In [9]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    block = block_combinations
    print(cities[l])
    # Check all parks within blockm radius
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    len3 = int(np.ceil(len2/block))
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per block
        l1, l2 = range(0,len1), range(i*block,(i+1)*block)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_proot2','size_infl_proot3','size_infl_proot5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_proot2'])
        infl3 = list(node_merged['size_infl_proot3'])
        infl5 = list(node_merged['size_infl_proot5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])

        if ((i+1) % int(np.floor(1000/block_combinations)) == 0) | ((i+1) == len3): 
            print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        if ((i+1) % int(np.floor(1000/block_combinations)) == 0) | ((i+1) == len3): 
            print('of',np.where(i+1 == len3, len2 % block * block, len1*block) ,
                                'within a Gravity model variant in one of',thresholds,'m threshold:',len(mat_df))

    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_proot2','size_infl_proot3','size_infl_proot5',
                      'parkshare_walked','park_area','walk_area_m2']
    
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_proot2','size_infl_proot3','size_infl_proot5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        try:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        except:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
                                                   ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]


Leiden
1 / 2 comb. done 0.14  mns
of 1090000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 134004
2 / 2 comb. done 0.18  mns
of 155000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 7272
total combinations within distance 0
0.0 % gridentry done 0.18  mns
100 % gridentry done 0.39  mns
Tashkent, Uzbekistan
1 / 2 comb. done 2.06  mns
of 13717000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 115523
2 / 2 comb. done 2.39  mns
of 36000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 3164
total combinations within distance 0
0.0 % gridentry done 2.41  mns
100 % gridentry done 2.61  mns
Providence, Rhode Island, United States
1 / 2 comb. done 2.78  mns
of 1608000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 74591
2 / 2 comb. done 2.82  mns
of 69000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 2836
total combinations within distance 0
0.0 % griden

,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_coords_centroid,Grid_m_centroid,Park_No,...,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,grid_osm,geometry,geometry_m,grid_entry_dist
0,36116,2371.962636,996.539812,1330.553628,1676.727810,146,33,POINT (4.48625 52.16292),POINT (601658.253 5780199.922),1,...,"POLYGON ((4.49495 52.13246, 4.49484 52.13254, ...",POINT (602011.508 5777854.412),301053.619820,2.380199,1.782688,1.414638,45525415,POINT (4.48601 52.16280),POINT (601642.056 5780186.966),20.741
1,41566,2297.358692,868.416245,1201.051312,1556.784362,146,38,POINT (4.48625 52.16292),POINT (601658.253 5780199.922),1,...,"POLYGON ((4.49495 52.13246, 4.49484 52.13254, ...",POINT (602306.761 5777995.995),371894.033477,2.645458,1.912790,1.475708,45525415,POINT (4.48601 52.16280),POINT (601642.056 5780186.966),20.741
2,42656,2296.392142,869.102096,1201.515052,1556.882906,146,39,POINT (4.48625 52.16292),POINT (601658.253 5780199.922),1,...,"POLYGON ((4.49495 52.13246, 4.49484 52.13254, ...",POINT (602310.707 5777998.168),370994.936520,2.642258,1.911247,1.474993,45525415,POINT (4.48601 52.16280),POINT (601642.056 5780186.966),20.741
3,45926,2304.575054,864.142728,1198.359911,1556.641863,146,42,POINT (4.48625 52.16292),POINT (601658.253 5780199.922),1,...,"POLYGON ((4.49495 52.13246, 4.49484 52.13254, ...",POINT (602281.327 5777981.173),377944.671605,2.666892,1.923108,1.480479,45525415,POINT (4.48601 52.16280),POINT (601642.056 5780186.966),20.741
4,57916,2263.339580,908.713051,1231.783597,1571.160446,146,53,POINT (4.48625 52.16292),POINT (601658.253 5780199.922),1,...,"POLYGON ((4.49495 52.13246, 4.49484 52.13254, ...",POINT (602414.876 5778066.796),329657.817052,2.490709,1.837449,1.440553,45525415,POINT (4.48601 52.16280),POINT (601642.056 5780186.966),20.741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141271,89374,939.615208,4506.208654,2672.129760,1759.116253,1084,1081,POINT (4.50792 52.16958),POINT (603124.736 5780971.952),49,...,"POLYGON ((4.49372 52.16732, 4.49424 52.16725, ...",POINT (602209.564 5780759.028),2310.441374,0.208516,0.351635,0.534140,296972150,POINT (4.50816 52.16953),POINT (603141.320 5780966.820),17.359
141272,90464,923.939346,4431.030322,2627.549877,1729.768428,1084,1082,POINT (4.50792 52.16958),POINT (603124.736 5780971.952),49,...,"POLYGON ((4.49372 52.16732, 4.49424 52.16725, ...",POINT (602222.632 5780772.271),2310.441374,0.208516,0.351635,0.534140,296972150,POINT (4.50816 52.16953),POINT (603141.320 5780966.820),17.359
141273,91554,946.101982,4537.317939,2690.577208,1771.260575,1084,1083,POINT (4.50792 52.16958),POINT (603124.736 5780971.952),49,...,"POLYGON ((4.49372 52.16732, 4.49424 52.16725, ...",POINT (602202.246 5780761.904),2310.441374,0.208516,0.351635,0.534140,296972150,POINT (4.50816 52.16953),POINT (603141.320 5780966.820),17.359
141274,158044,2392.428635,898.332824,1245.196361,1616.881911,1084,1144,POINT (4.50792 52.16958),POINT (603124.736 5780971.952),52,...,"POLYGON ((4.51057 52.19572, 4.51075 52.19584, ...",POINT (604013.279 5783193.260),376895.472262,2.663187,1.921326,1.479656,296972150,POINT (4.50816 52.16953),POINT (603141.320 5780966.820),17.359


In [10]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Leiden 141276
Tashkent, Uzbekistan 118687
Providence, Rhode Island, United States 77427


In [11]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = block_network

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    
    # Divide in chunks of block for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(-1)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                except:
                    # Otherwise the nearest node is taken, which is iterated 10 times at max. Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < nearest_node_iterations and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                        except:
                            try:
                                alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                    except:
                                        if len3 == nearest_node_iterations:
                                            print(i+block*k,'No route between grid and park-entry and both their 10 alternatives')
                                            pass
                                        pass
                    len4 = len(alt_route)
                    if len4 > 0: 
                        print('for index',i,'nearest node found between', 
                                               alt_route[0][0],'and',alt_route[0][-1])
                        
                        s_mat.append(alt_route[0])
                        shortest_to = list(alt_route[0][1:len(alt_route[0])])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4)))
                        s_mat3.append(list(np.arange(0, len4)))
                        s_mat4.append('altered way')
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),' mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # get single (dissolved) line per route, attach information.
        routes2 = routes[['route','geometry']].dissolve('route')
        routes2['way_calculated'] = s_mat4
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['num_steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid',
                                              'grid_osm','walk_area_m2','size_infl_proot2','size_infl_proot3',
                                              'size_infl_proot5','raw euclidean']],
                                                left_index = True, right_index = True)
        
        routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
        routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
        routes2['grav3_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot3']
        routes2['grav5_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]

Leiden 1 / 1 range 0 - 141276
0.0 % done 0.0 mns
7.08 % done 0.91 mns
14.16 % done 1.77 mns
21.24 % done 2.56 mns
28.31 % done 3.24 mns
35.39 % done 3.66 mns
42.47 % done 4.1 mns
49.55 % done 4.94 mns
56.63 % done 5.69 mns
63.71 % done 6.33 mns
70.78 % done 6.93 mns
77.86 % done 7.49 mns
84.94 % done 8.01 mns
92.02 % done 8.59 mns
99.1 % done 9.22 mns
100.0 % pathfinding done 9.27  mns
formatting done 10.67 mns
dissolving done 11.4 mns
Tashkent 1 / 1 range 0 - 118687
0.0 % done 11.41 mns
8.43 % done 11.62 mns
16.85 % done 11.89 mns
25.28 % done 12.08 mns
33.7 % done 12.27 mns
42.13 % done 12.53 mns
50.55 % done 12.87 mns
58.98 % done 13.03 mns
67.4 % done 13.27 mns
75.83 % done 13.58 mns
84.26 % done 14.07 mns
92.68 % done 14.37 mns
100.0 % pathfinding done 14.58  mns
formatting done 15.46 mns
dissolving done 15.98 mns
Providence 1 / 1 range 0 - 77427
0.0 % done 15.98 mns
12.92 % done 16.1 mns
25.83 % done 16.47 mns
38.75 % done 16.83 mns
51.66 % done 17.26 mns
64.58 % done 17.53 mns
7

,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,raw euclidean,raw_total_cost,grav2_total_cost,grav3_total_cost,grav5_total_cost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((4.48601 52.16280, 4.48581 52...",normal way,3642.378,56,146,1,33,20.741,32902427,45525415,301053.619820,2.380199,1.782688,1.414638,2371.962636,3663.119,1538.997227,2054.828437,2589.439397,146-1
1,"MULTILINESTRING ((4.48601 52.16280, 4.48581 52...",normal way,3314.037,54,146,1,38,20.741,262173364,45525415,371894.033477,2.645458,1.912790,1.475708,2297.358692,3334.778,1260.567364,1743.410599,2259.782183,146-1
2,"MULTILINESTRING ((4.48601 52.16280, 4.48581 52...",normal way,3309.543,53,146,1,39,20.741,262204268,45525415,370994.936520,2.642258,1.911247,1.474993,2296.392142,3330.284,1260.393098,1742.466489,2257.829635,146-1
3,"MULTILINESTRING ((4.48601 52.16280, 4.48581 52...",normal way,3343.405,55,146,1,42,20.741,2836013387,45525415,377944.671605,2.666892,1.923108,1.480479,2304.575054,3364.146,1261.448308,1749.328013,2272.336710,146-1
4,"MULTILINESTRING ((4.48601 52.16280, 4.48581 52...",normal way,3185.061,52,146,1,53,20.741,9403932806,45525415,329657.817052,2.490709,1.837449,1.440553,2263.339580,3205.802,1287.104305,1744.702542,2225.397083,146-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141271,"MULTILINESTRING ((4.50816 52.16953, 4.50819 52...",normal way,1370.428,18,1084,49,1081,17.359,9701826938,296972150,2310.441374,0.208516,0.351635,0.534140,939.615208,1387.787,6655.551909,3946.665522,2598.168535,1084-49
141272,"MULTILINESTRING ((4.50816 52.16953, 4.50819 52...",normal way,1343.245,17,1084,49,1082,17.359,9701826940,296972150,2310.441374,0.208516,0.351635,0.534140,923.939346,1360.604,6525.187619,3869.361001,2547.277429,1084-49
141273,"MULTILINESTRING ((4.50816 52.16953, 4.50819 52...",normal way,1373.620,18,1084,49,1083,17.359,9701826942,296972150,2310.441374,0.208516,0.351635,0.534140,946.101982,1390.979,6670.860110,3955.743108,2604.144491,1084-49


In [12]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Routes[n].iloc[Routes[n].groupby('gridpark_no')[(str(l1[i]) +'_total_cost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_total_cost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['population','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_total_cost'] <= t]
            thold[score] = t - thold[l1[i] + '_total_cost']
            thold['pop' + score] = thold[score] * thold['population']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['population'] / thold['walk_area_ha' + str2]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + str2]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2].sum()
            
            gs['walkha_person_' + score] = locals()[str1].groupby('Grid_No')['walkha_person' + str2].mean()

            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['population'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_lines/'):
                os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_lines/')
                
            gs.to_file('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_lines/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['population'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['population'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/'):
            os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/')
            
        gridsc.to_file('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_csv/'):
            os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_csv/')
        
        gridsc.to_csv('D:/Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/radius-euclidean adjusted popgrid access.csv')

popg_acc    


Leiden
entrance 0.01 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.12 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 0.23 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 0.31 mns
grid  300
grid  600
grid  1000
Leiden done 0.37 mns
Tashkent, Uzbekistan
entrance 0.38 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.86 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 1.33 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 1.79 mns
grid  300
grid  600
grid  1000
Tashkent, Uzbekistan done 2.26 mns
Providence, Rhode Island, United States
entrance 2.26 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 2.37 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 2.47 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 2.57 mns
grid  300
grid  600
grid  1000
Providence, Rhode Island, United States done 2.66 mns


Leiden  Tashkent, Uzbekistan  \
entrance_300        1 high    0.018507              0.000417   
                    2 medium  0.162137              0.008817   
                    3 low     0.183333              0.010769   
                    4 no      0.636023              0.979997   
entrance_600        1 high    0.135794              0.002118   
                    2 medium  0.286129              0.018730   
                    3 low     0.271008              0.030274   
                    4 no      0.307069              0.948878   
entrance_1000       1 high    0.428440              0.008966   
                    2 medium  0.290477              0.035009   
                    3 low     0.201048              0.061205   
                    4 no      0.080035              0.894819   
gravity**(1/2)_300  1 high    0.006603              0.000394   
                    2 medium  0.147068              0.013121   
                    3 low     0.182882              0.019095   
                    4 no      0.663447              0.967390   
gravity**(1/2)_600  1 high    0.070666              0.003157   
                    2 medium  0.303002              0.030721   
                    3 low     0.313961              0.054453   
                    4 no      0.312370              0.911670   
gravity**(1/2)_1000 1 high    0.382552              0.012256   
                    2 medium  0.319194              0.062933   
                    3 low     0.223240              0.127807   
                    4 no      0.075014              0.797003   
gravity**(1/3)_300  1 high    0.006390              0.000285   
                    2 medium  0.155024              0.011162   
                    3 low     0.174237              0.015043   
                    4 no      0.664349              0.973511   
gravity**(1/3)_600  1 high    0.071526              0.002445   
                    2 medium  0.293600              0.025083   
                    3 low     0.297752              0.042405   
                    4 no      0.337123              0.930067   
gravity**(1/3)_1000 1 high    0.316276              0.009289   
                    2 medium  0.351367              0.046956   
                    3 low     0.247890              0.088618   
                    4 no      0.084468              0.855137   
gravity**(1/5)_300  1 high    0.006390              0.000309   
                    2 medium  0.157747              0.009818   
                    3 low     0.180304              0.012976   
                    4 no      0.655560              0.976897   
gravity**(1/5)_600  1 high    0.091343              0.002218   
                    2 medium  0.292953              0.021811   
                    3 low     0.272736              0.033963   
                    4 no      0.342968              0.942008   
gravity**(1/5)_1000 1 high    0.336961              0.008738   
                    2 medium  0.343947              0.040450   
                    3 low     0.240053              0.069331   
                    4 no      0.079039              0.881482   

                              Providence, Rhode Island, United States  
entrance_300        1 high                                   0.010989  
                    2 medium                                 0.112684  
                    3 low                                    0.212160  
                    4 no                                     0.664167  
entrance_600        1 high                                   0.139021  
                    2 medium                                 0.280009  
                    3 low                                    0.324850  
                    4 no                                     0.256120  
entrance_1000       1 high                                   0.495989  
                    2 medium                                 0.288361  
                    3 low                                    0.143319  
                    4 no       

In [13]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'popreach'+score]] = str1.groupby('Park_No')[score,'pop' + score,'population'].sum()
            
            prk['walkha_person_' + score] = str1.groupby('Park_No')['walkha_person' + str2].mean()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
        
            if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_lines/'):
                os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_lines/')
            
            prk.to_file('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_lines/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['geometry'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_geoms/'):
            os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_geoms/')
            
        prksc.to_file('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_geoms/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_csv/'):
            os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_csv/')
            
        prksc.to_csv('D:/Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[0][0])

park 300
park 600
park 1000
entrance 0.03 mns
park 300
park 600
park 1000
gravity**(1/2) 0.08 mns
park 300
park 600
park 1000
gravity**(1/3) 0.11 mns
park 300
park 600
park 1000
gravity**(1/5) 0.13 mns
Leiden done 0.13 mns
park 300
park 600
park 1000
entrance 0.16 mns
park 300
park 600
park 1000
gravity**(1/2) 0.24 mns
park 300
park 600
park 1000
gravity**(1/3) 0.29 mns
park 300
park 600
park 1000
gravity**(1/5) 0.32 mns
Tashkent, Uzbekistan done 0.32 mns
park 300
park 600
park 1000
entrance 0.36 mns
park 300
park 600
park 1000
gravity**(1/2) 0.4 mns
park 300
park 600
park 1000
gravity**(1/3) 0.43 mns
park 300
park 600
park 1000
gravity**(1/5) 0.46 mns
Providence, Rhode Island, United States done 0.46 mns


,tr_300,pop_tr_300,popreachtr_300,walkha_person_tr_300,tr_300_parks,tr_300route,tr_300Grid:Pentry,tr_300_access,tr_600,pop_tr_600,...,tr_600Grid:Pentry,tr_600_access,tr_1000,pop_tr_1000,popreachtr_1000,walkha_person_tr_1000,tr_1000_parks,tr_1000route,tr_1000Grid:Pentry,tr_1000_access
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,3419.009,196948.690,1325.0,2.198824,23.0,"(LINESTRING (4.4980839 52.1437906, 4.4980132 5...","[649, 650, 682, 683, 684, 709, 710, 712, 713, ...",1 high,14381.393,840136.877,...,"[610, 649, 650, 651, 652, 676, 677, 679, 680, ...",1 high,51319.233,3253994.608,9419.0,2.929054,130.0,"(LINESTRING (4.4914206 52.1479287, 4.4917066 5...","[564, 565, 566, 571, 601, 602, 603, 605, 606, ...",1 high
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2287.526,186794.808,1385.0,7.504633,17.0,"(LINESTRING (4.5096768 52.1479997, 4.5096559 5...","[576, 577, 578, 614, 616, 617, 618, 619, 620, ...",1 high,9333.656,781348.631,...,"[532, 533, 534, 576, 577, 578, 579, 580, 581, ...",1 high,29032.008,2426503.846,6078.0,7.886961,71.0,"(LINESTRING (4.5130017 52.1547025, 4.5130798 5...","[378, 379, 426, 485, 486, 487, 488, 489, 490, ...",1 high
4,355.391,20238.833,117.0,15.416674,2.0,"(LINESTRING (4.4629473 52.1496003, 4.4629677 5...","[501, 502]:[85, 85]",3 low,2478.030,208588.935,...,"[451, 453, 454, 455, 456, 457, 458, 498, 499, ...",1 high,13653.436,1339246.019,5137.0,28.203509,48.0,"(LINESTRING (4.4558175 52.1544868, 4.4566075 5...","[341, 350, 386, 391, 395, 397, 398, 434, 435, ...",1 high
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,564.612,76762.657,535.0,51.296813,4.0,"(LINESTRING (4.4865885 52.1733774, 4.486963 52...","[1005, 1006, 976, 977]:[141, 137, 141, 140]",2 medium,3216.682,446876.965,...,"[1003, 1004, 1005, 1006, 1007, 1008, 1033, 103...",1 high,16549.812,2238615.886,6748.0,51.760881,50.0,"(LINESTRING (4.4812455 52.1727367, 4.4808646 5...","[1002, 1003, 1004, 1005, 1006, 1007, 1008, 103...",1 high
7,49.309,3846.102,78.0,5.219567,1.0,"(LINESTRING (4.4712968 52.1744572, 4.4713701 5...",[967]:[150],3 low,1169.905,106345.497,...,"[1024, 1053, 1054, 943, 967, 968, 969, 995, 99...",1 high,9619.170,839053.142,3188.0,4.785445,37.0,"(LINESTRING (4.4781835 52.1727406, 4.4766911 5...","[1000, 1024, 1025, 1026, 1027, 1028, 1029, 103...",1 high
8,4866.247,545163.834,3915.0,9.289915,34.0,"(LINESTRING (4.5028852 52.1830401, 4.5029334 5...","[4, 439, 440, 441, 442, 443, 444, 445, 5, 6, 8...",1 high,19750.568,2322083.123,...,"[10, 1016, 1017, 1018, 1019, 2, 3, 4, 438, 439...",1 high,51234.475,6039867.882,10551.0,9.381269,89.0,"(LINESTRING (4.4983419 52.1822732, 4.4983372 5...","[1, 10, 1012, 1013, 1014, 1015, 1016, 1017, 10...",1 high
9,2441.840,205741.516,1519.0,5.080475,18.0,"(LINESTRING (4.4812455 52.1727367, 4.4815932 5...","[1002, 923, 924, 925, 943, 944, 945, 946, 947,...",1 high,10183.265,910209.451,...,"[1000, 1001, 1002, 1003, 1004, 1006, 1029, 103...",1 high,29370.681,2843075.519,6659.0,6.983259,62.0,"(LINESTRING (4.4781835 52.1727406, 4.4766911 5...","[1000, 1001, 1002, 1003, 1004, 1005, 1006, 100...",1 high


In [14]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['dissolve_key'], how = 'outer')
            g_csv = g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')]
            
            if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/'):
                os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/')
                
            g_csv.to_csv('D:/Dumps/Scores output WP-OSM/'+str(grid_block_size)
                         +'m grids/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y'])], geometry = 'geometry_x', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/'):
                os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/')
                
            g_lines.to_file('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x'])], geometry = 'geometry_y', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/'):
                os.makedirs('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/')
                
            g_geoms.to_file('D:Dumps/Scores output WP-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')
preference[0][0][0]

Leiden
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
Tashkent, Uzbekistan
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
Providence, Rhode Island, United States
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
all done


,index,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,...,tr_600,poptr_600,adj_tr_600,walk_area_ha600,walkha_person600,tr_1000,poptr_1000,adj_tr_1000,walk_area_ha1000,walkha_person1000
4060,131646,"MULTILINESTRING ((4.50289 52.18304, 4.50293 52...",normal way,251.930,10,4,8,205,13.899,579974371,...,334.171,33082.929,334.171,8.555690,11.571247,734.171,72682.929,440.5026,8.555690,11.571247
4833,131753,"MULTILINESTRING ((4.50465 52.18291, 4.50416 52...",normal way,172.200,7,5,8,205,4.583,579974371,...,423.217,40205.615,423.217,8.555690,11.103721,823.217,78205.615,493.9302,8.555690,11.103721
5512,131829,"MULTILINESTRING ((4.50620 52.18295, 4.50606 52...",normal way,283.320,12,6,8,205,5.414,579974371,...,311.266,30192.802,311.266,8.555690,11.337484,711.266,68992.802,426.7596,8.555690,11.337484
2027,113720,"MULTILINESTRING ((4.46968 52.16800, 4.47018 52...",normal way,202.223,7,20,12,337,11.662,8560292393,...,386.115,50581.065,386.115,7.829879,16.730782,786.115,102981.065,471.6690,7.829879,16.730782
2099,113945,"MULTILINESTRING ((4.47107 52.16809, 4.47106 52...",normal way,96.620,3,21,12,328,23.217,5223854282,...,480.163,60500.538,480.163,7.797638,16.158739,880.163,110900.538,528.0978,7.797638,16.158739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,140733,"MULTILINESTRING ((4.50468 52.16891, 4.50451 52...",normal way,179.957,4,1082,27,633,75.020,249807250,...,345.023,28981.932,345.023,9.211589,9.118947,745.023,62581.932,447.0138,9.211589,9.118947
858,140967,None,normal way,0.000,0,1083,27,637,67.312,256322359,...,532.688,62324.496,532.688,6.580569,17.779618,932.688,109124.496,559.6128,6.580569,17.779618
871,141178,"MULTILINESTRING ((4.50816 52.16953, 4.50819 52...",normal way,243.309,5,1084,27,638,17.359,296972068,...,339.332,47167.148,339.332,6.743534,20.612338,739.332,102767.148,443.5992,6.743534,20.612338
882,88014,"MULTILINESTRING ((4.50993 52.16957, 4.50913 52...",normal way,231.373,5,1085,27,637,23.755,256322359,...,344.872,53455.160,344.872,6.580569,23.554194,744.872,115455.160,446.9232,6.580569,23.554194


In [15]:
print(round((time.time() - start) / 60,2),'mns')

31.3 mns
